In [39]:
import sklearn
import nltk
from sklearn.datasets import load_files
from nltk.corpus import movie_reviews
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

moviedir = r'C:\Users\aditt\AppData\Roaming\nltk_data\corpora\movie_reviews'
movie_train = load_files(moviedir, shuffle=True)


In [40]:
# initialize movie_vector object, and then turn movie train data into a vector 
movie_vec = CountVectorizer(min_df=2, tokenizer=nltk.word_tokenize)         # use all 25K words. 82.2% acc.
# movie_vec = CountVectorizer(min_df=2, tokenizer=nltk.word_tokenize, max_features = 3000) # use top 3000 words only. 78.5% acc.
movie_counts = movie_vec.fit_transform(movie_train.data)

In [41]:
tfidf_transformer = TfidfTransformer()
movie_tfidf = tfidf_transformer.fit_transform(movie_counts)
docs_train, docs_test, y_train, y_test = train_test_split(
    movie_train.data, movie_train.target, test_size = 0.20, random_state = 12)


In [42]:
import gensim
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.metrics import classification_report

w2v = Word2Vec(docs_train,vector_size = 1000, window = 5, workers = 4, min_count = 1,sg=1)
vocab = w2v.wv.key_to_index.keys()
opp = 0
average_list = []
train_vec = []
for i in docs_train:
# this is a good product 5 ---> 1

        if opp == 29:
            opp = 29
        for token in i:
          if token in vocab:
            average_list.append(w2v.wv.get_vector(token))

          else:
            average_list.append(np.zeros(1000))

        train_vec.append(np.mean(average_list, axis = 0).tolist())
        opp += 1
        average_list = []

j = 0
for i in train_vec:
    if type(i) is list:
        j+=1
        continue
    elif np.isnan(i):
        train_vec[j] = [0] * 1000
    j += 1

wmod = LogisticRegression(max_iter=10000, multi_class='multinomial',solver='lbfgs',class_weight='balanced')
wmod.fit(train_vec, y_train)

test_vec = []

for i in docs_test:

        for token in i:
          if token in vocab:
            average_list.append(w2v.wv.get_vector(token))
          else:
            average_list.append(np.zeros(1000))

        test_vec.append(np.mean(average_list, axis = 0).tolist())
        opp += 1
        average_list = []

j = 0
for i in test_vec:
    if type(i) is list:
        j+=1
        continue
    elif np.isnan(i):
        test_vec[j] = [0] * 1000
    j += 1

y_pred = wmod.predict(test_vec)
print(classification_report(y_test,y_pred))


              precision    recall  f1-score   support

           0       0.60      0.57      0.58       206
           1       0.56      0.59      0.58       194

    accuracy                           0.58       400
   macro avg       0.58      0.58      0.58       400
weighted avg       0.58      0.58      0.58       400

